In [1]:
!pip3 install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=867234 sha256=a5f5007337a4bc5a7e31b52c201656e963db3f3a2103f5a730478f28b6b9d4d7
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
sent_model = hub.load(module_url)
print ("module %s loaded" % module_url)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
def embed(input):
  return sent_model(input)

In [4]:
# Load item information
items = pd.read_csv('/content/drive/MyDrive/jester_items.csv')
# Load user ratings
ratings = pd.read_excel('/content/drive/MyDrive/jester-data-1.xls', header=None)
# Remove the first column containing the number of jokes rated by that user.
ratings.drop(columns=0, inplace=True)
ratings = ratings.T
ratings.columns = ["user_" + str(col) for col in ratings.columns]

# Convert the ratings below -10 to -10 and above 10 to 10
ratings[ratings < -10] = -10
ratings[ratings > 10] = 10



df_normalized = ratings.applymap(lambda x: 0 if x == 99 else x )


# Load item information
items = pd.read_csv('/content/drive/MyDrive/jester_items.csv')

In [5]:

# Create item features
item_list = []
# Encode jokes with Universal Sentence Encoder
# item_features = embed(items.iloc[:, 1].values).numpy()

for joke in items.iloc[:, 1].values:
    embeddings = embed([joke])
    item_list.append(embeddings[0])

In [6]:
np_items = df_normalized.T.columns.values.astype(int)
np_users = df_normalized.columns.values.astype(str)


dataset = Dataset(item_identity_features=False)
dataset.fit(users=df_normalized.columns.values, items=df_normalized.T.columns.values,
                         item_features=list(range(len(item_list[0]))))

(interactions, weights) = dataset.build_interactions(((k, v)
                                                                             for k, v in
                                                                             zip(list(df_normalized.columns.values),
                                                                                 list(df_normalized.T.columns.values))))

In [7]:
item_features = dataset.build_item_features(
    ((k, {fn: fv for fn, fv in enumerate(v)}) for k, v in zip(df_normalized.T.columns.values, item_list)), normalize=False)


In [8]:
model = LightFM(loss='warp')
model.fit(interactions, item_features=item_features,
                epochs=5)

In [9]:
user_id = "user_943"
scores = model.predict(dataset.mapping()[0][user_id],
                                    np.arange(dataset.interactions_shape()[1]))

In [10]:
recommendations = items['jokeText'].loc[np_items[np.argsort(-scores)][:5]]

print(f"Recommendations for  {user_id} \n{recommendations} ")

Recommendations for  user_943 
33    Out in the backwoods of some midwestern state,...
36    A Jewish young man was seeing a psychiatrist f...
38    What is the difference between men and women:\...
64    Two Rednecks were seated at the end of a bar w...
85    A neutron walks into a bar and orders a drink....
Name: jokeText, dtype: object 
